# S1S2 / S3 Temperature Score Calculation

This notebook provides a comparison between temparature calculation for scopes S1S2 and S3

## Getting started
Make sure you are running the notebook with the requirements installed available in the example folder.

If you see errors when attempting to load the ITR modules, go to the top-level ITR directory, activate the `itr_env` conda environment (using `conda activate itr_env` and execute the command `pip install -e .`.  Then try again, or hit the <i class="fas fa-forward"></i> button above.

In [1]:
import os
import sys
import warnings

If all is well, the list of paths below will prioritize loading from the environment established for the ITR tool.  Please contact us if not.

In [2]:
display(sys.path)

['/Users/michael/Documents/GitHub/MichaelTiemannOSC/ITR/examples',
 '/Users/michael/Documents/GitHub/MichaelTiemannOSC/ITR/examples',
 '/Library/Application Support/Blackmagic Design/DaVinci Resolve/Developer/Scripting/Modules',
 '/Users/michael/opt/miniconda3/envs/itr_env/lib/python39.zip',
 '/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9',
 '/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/lib-dynload',
 '',
 '/Users/michael/.local/lib/python3.9/site-packages',
 '/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/site-packages',
 '/Users/michael/Documents/GitHub/MichaelTiemannOSC/pint',
 '/Users/michael/Documents/GitHub/MichaelTiemannOSC/pint-pandas',
 '/Users/michael/Documents/GitHub/globalwarmingpotentials/src',
 '/Users/michael/Documents/GitHub/MichaelTiemannOSC/ITR']

In [3]:
# Load the ITR library files

import ITR
from ITR.data.excel import ExcelProviderProductionBenchmark, ExcelProviderIntensityBenchmark
from ITR.data.template import TemplateProviderCompany
from ITR.data.base_providers import BaseProviderProductionBenchmark, BaseProviderIntensityBenchmark
from ITR.data.data_warehouse import DataWarehouse
from ITR.portfolio_aggregation import PortfolioAggregationMethod
from ITR.temperature_score import TemperatureScore
from ITR.interfaces import ETimeFrames, EScope, IProductionBenchmarkScopes, IEIBenchmarkScopes
import pandas as pd

from ITR.data.osc_units import ureg, Q_, PA_, requantify_df_from_columns

In [4]:
print("Testing unit registry\n=====================")
one_co2 = ureg("CO2e")
print(f"The gas species CO2e, which was a gwp of 1: {one_co2}")

one_Gt_co2 = ureg("Gt CO2e")
print(f"A gigaton of CO2e: {one_Gt_co2}")

Testing unit registry
The gas species CO2e, which was a gwp of 1: 1 CO2e
A gigaton of CO2e: 1 CO2e * gigametric_ton


## Load the production and intensity benchmarks

Prepare the various benchmark files; the OECM benchmark is the default after the next cell finishes.

In [5]:
import json

self_root = os.path.abspath('')
benchmark_prod_json = os.path.join(self_root, "data", "json-units", "benchmark_production_OECM.json")
benchmark_EI_OECM_S1S2 = os.path.join(self_root, "data", "json-units", "benchmark_EI_OECM_PC.json")
benchmark_EI_OECM_S3 = os.path.join(self_root, "data", "json-units", "benchmark_EI_OECM_S3.json")

# load production benchmarks
with open(benchmark_prod_json) as json_file:
    parsed_json = json.load(json_file)
prod_bms = IProductionBenchmarkScopes.model_validate(parsed_json)
base_production_bm = BaseProviderProductionBenchmark(production_benchmarks=prod_bms)

# load intensity benchmarks

# OECM for scope S1S2
with open(benchmark_EI_OECM_S1S2) as json_file:
    parsed_json = json.load(json_file)
ei_bms_s1s2 = IEIBenchmarkScopes.model_validate(parsed_json)
OECM_EI_bm_s1s2 = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms_s1s2)

# OECM for scope S3
with open(benchmark_EI_OECM_S3) as json_file:
    parsed_json = json.load(json_file)
ei_bms_s3 = IEIBenchmarkScopes.model_validate(parsed_json)
OECM_EI_bm_s3 = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms_s3)

base_intensity_bm_s1s2 = OECM_EI_bm_s1s2
base_intensity_bm_s3 = OECM_EI_bm_s3

## Download/load the sample template data


In [6]:
import urllib.request

if not os.path.isdir("data"):
    os.mkdir("data")

for filename in ['data/20220927 ITR Tool Sample Data.xlsx',
                 'data/OECM_EI_and_production_benchmarks.xlsx',
                 'utils.py']:
    if not os.path.isfile(filename):
        # Get a specific content file:
        contents = repo.get_contents(f"examples/{filename}")

        # Donwnload file form ContenFile object info:
        urllib.urlretrieve(contents.download_url, filename)

try:  # Import statement when run in remote Jupyter servers from AWS Google etc..
    from utils import collect_company_contributions, plot_grouped_statistics, anonymize, \
        plot_grouped_heatmap, print_grouped_scores, get_contributions_per_group
except:  # Import statement when run locally
    from utils import collect_company_contributions, plot_grouped_statistics, anonymize, \
        plot_grouped_heatmap, print_grouped_scores, get_contributions_per_group

template_data_path = "data/20220927 ITR Tool Sample Data.xlsx"

### Logging
The ITR module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [7]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider

In [8]:
# Remove the # and space on the next line to point the template_data_path variable at your own data
# template_data_path = "data/your_template_here.xlsx"

template_company_data = TemplateProviderCompany(excel_path=template_data_path)

2023-06-08 20:24:12,361 - ITR.data.template - WARNING - Missing target start year set to 2021 for companies with ID: ['US0185223007', 'US0188021085', 'US0236081024', 'US0255371017', 'US05351W1036-E', 'US05351W1036-U', 'US0921131092', 'US1442851036', 'US18551QAA58', 'US25746U1097', 'US26441C2044', 'US30034W1062', 'US5526901096', 'JP3633400001']


## Create the Data Warehouse

In [9]:
template_company_data._companies[0].production_metric

ProductionMetric('GWh')

In [10]:

print(f"S1S2:\nBenchmark Temperature = {base_intensity_bm_s1s2.benchmark_temperature}\n\
Benchmark Global Budget = {base_intensity_bm_s1s2.benchmark_global_budget}\n\
AFOLU included = {base_intensity_bm_s1s2.is_AFOLU_included}")

template_provider_s3 = DataWarehouse(template_company_data, base_production_bm, base_intensity_bm_s3)
print(f"S3:\nBenchmark Temperature = {base_intensity_bm_s3.benchmark_temperature}\n\
Benchmark Global Budget = {base_intensity_bm_s3.benchmark_global_budget}\n\
AFOLU included = {base_intensity_bm_s3.is_AFOLU_included}")

2023-06-08 20:24:19,109 - ITR.data.data_warehouse - INFO - new_production_bm calculating trajectories for 76 companies (times 5 scopes times 31 years)


S1S2:
Benchmark Temperature = 1.5 delta_degree_Celsius
Benchmark Global Budget = 521.0526315789474 CO2 * gigametric_ton
AFOLU included = False


2023-06-08 20:24:29,488 - ITR.data.base_providers - INFO - Normalizing intensity metrics
2023-06-08 20:24:29,776 - ITR.data.base_providers - INFO - Done normalizing intensity metrics
2023-06-08 20:24:29,776 - ITR.data.data_warehouse - INFO - projecting targets for 76 companies (times 5 scopes times 31 years)


S3:
Benchmark Temperature = 1.5 delta_degree_Celsius
Benchmark Global Budget = 521.0526315789474 CO2 * gigametric_ton
AFOLU included = False


## Load your portfolio

The portfolio data is a sheet in the Data Template named "Portfolio".

Please see the technical documentation in the [Data Template Requirements](https://github.com/os-c/ITR/blob/main/docs/DataTemplateRequirements.rst) section for details on data requirements.

In [11]:
df_portfolio = requantify_df_from_columns(pd.read_excel(template_data_path, sheet_name="Portfolio"))
display(df_portfolio.tail())

,company_name,company_lei,company_id,company_isin,investment_value
71,WEC Energy Group,549300IGLYTZUK3PVP70,US92939U1060,US92939U1060,194900720.0
72,WORTHINGTON INDUSTRIES INC,1WRCIANKYOIK6KYE5E82,US9818111026,US9818111026,250005145.0
73,"Xcel Energy, Inc.",LGJNMI9GH8XIDG5RCM61,US98389B1008,US98389B1008,92453214.0
74,Balfour Beatty,CT4UIJ3TUKGYYHMENQ17,GB0000961622,GB0000961622,139657677.0
75,CBRE,52990016II9MJ2OSWA10,US12504L1098,US12504L1098,142575850.0


To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://ofbdabv.github.io/ITR/autoapi/ITR/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [12]:
companies = ITR.utils.dataframe_to_portfolio(df_portfolio)

## Calculate the temperature scores
In the enhanced portfolio you'll find your original portfolio, with calculated temperature scores, trajectory and target scores and overshoot/undershoot ration, and a temperature_result which is current set to zero for all valid calculations.

In [13]:
template_provider = DataWarehouse(template_company_data, base_production_bm, base_intensity_bm_s1s2)
temperature_score_s1s2 = TemperatureScore(
    time_frames = [ETimeFrames.LONG],
    scopes=[EScope.S1S2],
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
enhanced_portfolio_s1s2 = temperature_score_s1s2.calculate(data_warehouse=template_provider, portfolio=companies)

temperature_score_s3 = TemperatureScore(
    time_frames = [ETimeFrames.LONG],
    scopes=[EScope.S3],
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
template_provider.update_benchmarks(base_production_bm, base_intensity_bm_s3)
enhanced_portfolio_s3 = temperature_score_s3.calculate(data_warehouse=template_provider, portfolio=companies)

2023-06-08 20:24:36,294 - ITR.data.data_warehouse - INFO - new_production_bm calculating trajectories for 76 companies (times 5 scopes times 31 years)
2023-06-08 20:24:36,296 - ITR.data.base_providers - INFO - Normalizing intensity metrics
2023-06-08 20:24:36,549 - ITR.data.base_providers - INFO - Done normalizing intensity metrics
2023-06-08 20:24:36,550 - ITR.data.data_warehouse - INFO - projecting targets for 76 companies (times 5 scopes times 31 years)
2023-06-08 20:24:36,550 - ITR.data.data_warehouse - INFO - Shifting S3 emissions data into S1 according to Production-Centric benchmark rules
2023-06-08 20:24:37,317 - ITR.temperature_score - INFO - calculating temperature score for 76 companies
2023-06-08 20:24:40,574 - ITR.temperature_score - INFO - temperature score preparing data
2023-06-08 20:24:40,713 - ITR.temperature_score - INFO - temperature score data prepared
2023-06-08 20:24:41,036 - ITR.temperature_score - INFO - calculating temperature score for 76 companies
2023-06-08

For every company the tool assigns a score for all the requested timeframe and scope combinations.

In [14]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    display(enhanced_portfolio_s1s2[['company_name', 'time_frame', 'scope', 'temperature_score']])

,company_name,time_frame,scope,temperature_score
company_id,,,,
US00130H1059,AES Corp.,LONG,S1S2,2.9794263427297234+/-0
US0185223007,"ALLETE, Inc.",LONG,S1S2,2.72598714093074+/-0
US0188021085,Alliant Energy,LONG,S1S2,2.083634129448012+/-0
US0236081024,Ameren Corp.,LONG,S1S2,2.23110802721943+/-0
US0255371017,"American Electric Power Co., Inc.",LONG,S1S2,2.8703555008539263+/-0
...,...,...,...,...
US92939U1060,WEC Energy Group,LONG,S1S2,2.5132342397389724+/-0
US9818111026,WORTHINGTON INDUSTRIES INC,LONG,S1S2,1.1990033955102688+/-0
US98389B1008,"Xcel Energy, Inc.",LONG,S1S2,2.1565905278243367+/-0


In [15]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    display(enhanced_portfolio_s3[['company_name', 'time_frame', 'scope', 'temperature_score']])

,company_name,time_frame,scope,temperature_score
company_id,,,,
US00130H1059,AES Corp.,LONG,S3,3.2+/-0
US05351W1036-E,"Avangrid, Inc. (Elec)",LONG,S3,3.2+/-0
US05351W1036-U,"Avangrid, Inc. (Util)",LONG,S3,2.3251027888551468+/-0
DE0005190003,BMW Group,LONG,S3,1.8172933366737691+/-0
GB0007980591,BP plc,LONG,S3,1.5715163257740852+/-0
US1667641005,Chevron Corp,LONG,S3,1.5349620591256452+/-0
US1858991011,CLEVELAND-CLIFFS INC,LONG,S3,1.208265629360464+/-0
FR0010242511,Electricité de France,LONG,S3,3.2+/-0
US2810201077,Edison International,LONG,S3,3.2+/-0
